In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Santa Catarina - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [6]:
data = pd.read_csv('2003_01_model_input_SC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Consumo de Cimento (t)
0,2003,226.842795,8.256606,0.797768,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,...,1186.489105,20.555188,15.857813,74.759098,2.134406,1.320065e+08,6.638363e+06,19.725168,1.221860e+08,1687.172000
1,2004,221.696363,8.186110,0.798794,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,...,1168.021767,20.546378,15.775556,74.696472,2.223296,1.340197e+08,6.682964e+06,19.810475,1.229692e+08,1740.105000
2,2005,223.704012,8.161534,0.799746,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,...,1184.955322,21.126391,16.243994,74.614798,2.321951,1.358571e+08,6.722583e+06,19.879681,1.235946e+08,1824.650000
3,2006,265.456919,8.074449,0.800627,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,...,1181.222754,21.054599,16.370681,75.642600,2.275732,1.375869e+08,6.771741e+06,19.950366,1.241926e+08,1870.478000
4,2007,322.873579,8.049751,0.801440,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,...,1175.578011,20.950887,16.605682,77.293085,2.233296,1.391380e+08,6.821246e+06,20.007428,1.246727e+08,2081.839000
5,2008,354.241699,8.020915,0.802185,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,...,1159.944051,20.992808,16.885552,78.403820,2.202312,1.404443e+08,6.811481e+06,20.042997,1.249726e+08,2653.587000
6,2009,431.347926,8.049936,0.802866,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,...,1144.653844,20.979645,16.992954,78.934674,2.170937,1.414019e+08,6.842789e+06,20.044049,1.250753e+08,2669.400000
7,2010,792.441715,8.013741,0.803485,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,...,1157.121042,20.848277,16.799836,78.648358,2.053981,1.450816e+08,6.897548e+06,20.092849,1.259937e+08,2852.759000
8,2011,1268.083588,8.000656,0.804044,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,...,1175.335805,20.915534,16.747690,78.133817,1.801280,1.483956e+08,6.888268e+06,20.118967,1.266496e+08,3087.810000
9,2012,1549.260439,4.760238,0.804247,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,...,1187.050216,21.084303,16.909717,78.138285,1.427377,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,3268.310000


In [7]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,226.842795,8.256606,0.797768,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,1021.831952,1186.489105,20.555188,15.857813,74.759098,2.134406,1.320065e+08,6.638363e+06,19.725168,1.221860e+08
1,221.696363,8.186110,0.798794,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,1020.654104,1168.021767,20.546378,15.775556,74.696472,2.223296,1.340197e+08,6.682964e+06,19.810475,1.229692e+08
2,223.704012,8.161534,0.799746,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,1017.740527,1184.955322,21.126391,16.243994,74.614798,2.321951,1.358571e+08,6.722583e+06,19.879681,1.235946e+08
3,265.456919,8.074449,0.800627,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,1016.735433,1181.222754,21.054599,16.370681,75.642600,2.275732,1.375869e+08,6.771741e+06,19.950366,1.241926e+08
4,322.873579,8.049751,0.801440,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,1016.059668,1175.578011,20.950887,16.605682,77.293085,2.233296,1.391380e+08,6.821246e+06,20.007428,1.246727e+08
5,354.241699,8.020915,0.802185,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,1015.714984,1159.944051,20.992808,16.885552,78.403820,2.202312,1.404443e+08,6.811481e+06,20.042997,1.249726e+08
6,431.347926,8.049936,0.802866,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,1015.471175,1144.653844,20.979645,16.992954,78.934674,2.170937,1.414019e+08,6.842789e+06,20.044049,1.250753e+08
7,792.441715,8.013741,0.803485,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,1015.574102,1157.121042,20.848277,16.799836,78.648358,2.053981,1.450816e+08,6.897548e+06,20.092849,1.259937e+08
8,1268.083588,8.000656,0.804044,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,1015.697189,1175.335805,20.915534,16.747690,78.133817,1.801280,1.483956e+08,6.888268e+06,20.118967,1.266496e+08
9,1549.260439,4.760238,0.804247,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,1015.790103,1187.050216,21.084303,16.909717,78.138285,1.427377,1.513674e+08,6.873501e+06,20.117079,1.271489e+08


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1740.105000
1     1824.650000
2     1870.478000
3     2081.839000
4     2653.587000
5     2669.400000
6     2852.759000
7     3087.810000
8     3268.310000
9     3466.227000
10    3246.749286
11    3017.084229
12    2776.217486
13    2891.489000
14    2976.485000
15    3258.027000
16    3537.032000
17    4089.333000
18    4255.808000
19            NaN
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,-1.244844,1.052493,-1.994285,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,...,2.941174,1.136445,-1.946930,-2.035890,-1.569926,0.655800,-1.620438,-2.186864,-2.677515,-2.020278
1,-1.253591,1.000393,-1.629028,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,...,2.271547,-0.147397,-1.978422,-2.228888,-1.611788,0.905955,-1.410301,-1.538165,-1.877756,-1.604579
2,-1.250179,0.982230,-1.289984,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,...,0.615128,1.029817,0.095092,-1.129801,-1.666381,1.183592,-1.218502,-0.961918,-1.228940,-1.272609
3,-1.179217,0.917871,-0.976329,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,...,0.043714,0.770330,-0.161562,-0.832560,-0.979367,1.053520,-1.037936,-0.246922,-0.566262,-0.955143
4,-1.081632,0.899618,-0.687238,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,...,-0.340470,0.377910,-0.532327,-0.281181,0.123868,0.934097,-0.876030,0.473105,-0.031300,-0.700315
5,-1.028320,0.878307,-0.421888,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,...,-0.536429,-0.708957,-0.382461,0.375472,0.866316,0.846901,-0.739679,0.331079,0.302166,-0.541142
6,-0.897272,0.899755,-0.179454,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,...,-0.675039,-1.771926,-0.429516,0.627465,1.221155,0.758604,-0.639717,0.786439,0.312029,-0.486628
7,-0.283564,0.873005,0.040888,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,...,-0.616523,-0.905211,-0.899150,0.174358,1.029773,0.429465,-0.255618,1.582900,0.769536,0.000860
8,0.524827,0.863335,0.239962,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,...,-0.546546,0.361072,-0.658712,0.052008,0.685838,-0.281692,0.090305,1.447916,1.014396,0.349023
9,1.002709,-1.531473,0.312322,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,...,-0.493723,1.175453,-0.055369,0.432168,0.688825,-1.333939,0.400513,1.233137,0.996692,0.614069


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1740.105000
1     1824.650000
2     1870.478000
3     2081.839000
4     2653.587000
5     2669.400000
6     2852.759000
7     3087.810000
8     3268.310000
9     3466.227000
10    3246.749286
11    3017.084229
12    2776.217486
13    2891.489000
14    2976.485000
15    3258.027000
16    3537.032000
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
17,1.271362,-1.092682,-0.11741,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.364725,0.273868,0.225803,-0.51059,-0.762042,-0.252919,0.886688,-0.271847,-0.02587,0.728476


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    4089.333
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [13]:
train_input.iloc[-(len(train_input)//5):]

,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
14,0.805203,-0.965102,1.300358,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,...,-0.357193,-0.433575,1.336772,0.718249,-0.093965,-0.081732,1.217713,-0.707742,0.344885,1.039726
15,0.856363,-1.045743,0.977481,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,...,-0.335593,0.328728,1.420117,0.328890,-0.675911,0.157206,1.214451,-0.532297,0.389403,1.041875
16,1.076131,-1.100971,0.560529,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,...,-0.333901,0.829324,0.890486,-0.214901,-0.935987,0.114395,1.131669,-0.416908,0.288784,0.974515


In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [15]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2048416779, 502928877, 1366093472, 3093772077, 3908415912, 822852585, 2684582651, 1601900709, 3159054158, 2628440793, 3552838759, 724661411, 1712856046, 3655706504, 2831509464, 3020293211, 825902018, 3407845406, 962912566, 638594645, 1159671305, 1867996965, 2717140979, 482615389, 2639531762, 2197755632, 2952360540, 571366110, 1681146615, 3097126531, 2320500899, 1091801085, 1259422426, 1878969801, 3334100103, 1623716403, 3356733998, 2368917766, 1711579737, 370541962, 1573222163, 2706029796, 2632083518, 112200797, 2791612801, 1883382490, 2035828679, 1117447913, 3008017461, 2621116691, 3725218570, 1236405311, 425317537, 3657961814, 3867559277, 1291610263, 2982395683, 18051058, 2593777139, 4131851861, 387043051, 1014307009, 2224972656, 1222212422, 2268855712, 3256994025, 3210055085, 1384673773, 463225015, 3782964296, 1623577224, 3248165935, 94367936, 2295212892, 2789948722, 223636265, 1331512297, 123518221, 924399808, 2615967124, 920084999, 3363175903, 977694478, 921548183, 1107708019, 38

val_loss: 52369.1875


Step: 88 ___________________________________________
val_loss: 52372.30078125


Step: 89 ___________________________________________
val_loss: 52843.52734375


Step: 90 ___________________________________________
val_loss: 92588.8125


Step: 91 ___________________________________________
val_loss: 56147.50390625


Step: 92 ___________________________________________
val_loss: 36615.83203125


Step: 93 ___________________________________________
val_loss: 61463.78125


Step: 94 ___________________________________________
val_loss: 52394.51171875


Step: 95 ___________________________________________
val_loss: 47769.46484375


Step: 96 ___________________________________________
val_loss: 132850.625


Step: 97 ___________________________________________
val_loss: 54506.88671875


Step: 98 ___________________________________________
val_loss: 52323.03125


Step: 99 ___________________________________________
val_loss: 52391.94921875


final_seed: 1867996965


In [18]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 476ms/step - loss: 8025302.5000 - val_loss: 10448663.0000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 7865131.5000 - val_loss: 8183436.0000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 7381190.0000 - val_loss: 8237942.5000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 7127436.0000 - val_loss: 9060553.0000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 6631835.0000 - val_loss: 8446660.0000
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 7176775.5000 - val_loss: 10933126.0000
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 6869493.0000 - val_loss: 10230537.0000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 7746431.5000 - val_loss: 11986512.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 7225452.0000 - val_loss: 

1/1 [==============================] - 0s 14ms/step - loss: 2074303.2500 - val_loss: 2113271.7500
Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 1979830.6250 - val_loss: 2463395.7500
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 1742899.0000 - val_loss: 1980986.6250
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 1713513.7500 - val_loss: 1956208.6250
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 1905822.5000 - val_loss: 2159609.7500
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 1602253.2500 - val_loss: 1586659.3750
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 1702272.7500 - val_loss: 2013478.1250
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 1728502.6250 - val_loss: 1709907.0000
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 1829473.6250 - val_loss: 2132444.250

Epoch 146/10000
1/1 [==============================] - 0s 14ms/step - loss: 546822.1250 - val_loss: 637079.9375
Epoch 147/10000
1/1 [==============================] - 0s 14ms/step - loss: 634133.6875 - val_loss: 684112.9375
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 586924.6875 - val_loss: 511388.5938
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 598033.1250 - val_loss: 666539.4375
Epoch 150/10000
1/1 [==============================] - 0s 15ms/step - loss: 528007.6250 - val_loss: 496740.2812
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 501319.2500 - val_loss: 446581.9688
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 539056.1875 - val_loss: 578044.2500
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 555911.0625 - val_loss: 495889.4688
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 524916.3750 - val_loss: 4714

1/1 [==============================] - 0s 14ms/step - loss: 284217.3750 - val_loss: 258905.6406
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 188216.9062 - val_loss: 206328.5156
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 326977.2812 - val_loss: 254689.6719
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 264055.5625 - val_loss: 166297.7031
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 269935.2812 - val_loss: 196059.9375
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 242796.2969 - val_loss: 163994.0156
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 266031.8125 - val_loss: 303631.1250
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 298545.6562 - val_loss: 165667.4844
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 207989.5312 - val_loss: 208390.2500
Epoch 22

1/1 [==============================] - 0s 15ms/step - loss: 180753.0938 - val_loss: 135537.8281
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 155662.0312 - val_loss: 101858.9453
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 172093.5781 - val_loss: 123271.3984
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 181346.8750 - val_loss: 136098.1719
Epoch 296/10000
1/1 [==============================] - 0s 14ms/step - loss: 162385.1250 - val_loss: 101015.6172
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 189557.9062 - val_loss: 96794.9375
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 163711.1094 - val_loss: 126282.9766
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 159649.1875 - val_loss: 119200.1797
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 172397.6406 - val_loss: 87520.8516
Epoch 301/

...

Epoch 736/10000
1/1 [==============================] - 0s 14ms/step - loss: 149012.0156 - val_loss: 57700.2969
Epoch 737/10000
1/1 [==============================] - 0s 14ms/step - loss: 143899.4531 - val_loss: 53765.9180
Epoch 738/10000
1/1 [==============================] - 0s 13ms/step - loss: 124873.3516 - val_loss: 52393.0117
Epoch 739/10000
1/1 [==============================] - 0s 14ms/step - loss: 115514.0469 - val_loss: 53985.4688
Epoch 740/10000
1/1 [==============================] - 0s 14ms/step - loss: 126509.0000 - val_loss: 54320.5508
Epoch 741/10000
1/1 [==============================] - 0s 14ms/step - loss: 119305.2188 - val_loss: 53920.4883
Epoch 742/10000
1/1 [==============================] - 0s 14ms/step - loss: 153064.8750 - val_loss: 60663.6250
Epoch 743/10000
1/1 [==============================] - 0s 14ms/step - loss: 126674.2500 - val_loss: 58054.4102
Epoch 744/10000
1/1 [==============================] - 0s 14ms/step - loss: 112899.8203 - val_loss: 52904.7500
E

Epoch 810/10000
1/1 [==============================] - 0s 14ms/step - loss: 111436.7500 - val_loss: 53643.4023
Epoch 811/10000
1/1 [==============================] - 0s 14ms/step - loss: 182237.7812 - val_loss: 55138.0625
Epoch 812/10000
1/1 [==============================] - 0s 14ms/step - loss: 120761.6562 - val_loss: 53132.3945
Epoch 813/10000
1/1 [==============================] - 0s 14ms/step - loss: 111112.7891 - val_loss: 54583.5977
Epoch 814/10000
1/1 [==============================] - 0s 13ms/step - loss: 147812.8750 - val_loss: 66181.7188
Epoch 815/10000
1/1 [==============================] - 0s 14ms/step - loss: 140219.3281 - val_loss: 54435.1445
Epoch 816/10000
1/1 [==============================] - 0s 14ms/step - loss: 158382.3750 - val_loss: 54221.5195
Epoch 817/10000
1/1 [==============================] - 0s 14ms/step - loss: 126130.9297 - val_loss: 54147.3164
Epoch 818/10000
1/1 [==============================] - 0s 14ms/step - loss: 146796.8750 - val_loss: 52835.4258
E

Epoch 884/10000
1/1 [==============================] - 0s 14ms/step - loss: 147460.5938 - val_loss: 57272.1367
Epoch 885/10000
1/1 [==============================] - 0s 14ms/step - loss: 120458.3359 - val_loss: 55689.0117
Epoch 886/10000
1/1 [==============================] - 0s 14ms/step - loss: 157995.2969 - val_loss: 59179.2773
Epoch 887/10000
1/1 [==============================] - 0s 14ms/step - loss: 118837.9141 - val_loss: 52372.3242
Epoch 888/10000
1/1 [==============================] - 0s 14ms/step - loss: 164651.2188 - val_loss: 61744.9844
Epoch 889/10000
1/1 [==============================] - 0s 14ms/step - loss: 145778.1094 - val_loss: 53054.7305
Epoch 890/10000
1/1 [==============================] - 0s 14ms/step - loss: 114276.8906 - val_loss: 53281.6367
Epoch 891/10000
1/1 [==============================] - 0s 14ms/step - loss: 117251.9219 - val_loss: 61487.2617
Epoch 892/10000
1/1 [==============================] - 0s 14ms/step - loss: 127630.5469 - val_loss: 63781.5625
E

Epoch 958/10000
1/1 [==============================] - 0s 14ms/step - loss: 120572.6094 - val_loss: 57780.7812
Epoch 959/10000
1/1 [==============================] - 0s 14ms/step - loss: 153617.5000 - val_loss: 59164.4180
Epoch 960/10000
1/1 [==============================] - 0s 14ms/step - loss: 143626.6562 - val_loss: 67510.7109
Epoch 961/10000
1/1 [==============================] - 0s 14ms/step - loss: 164813.5312 - val_loss: 52819.0117
Epoch 962/10000
1/1 [==============================] - 0s 14ms/step - loss: 176227.0781 - val_loss: 57080.1250
Epoch 963/10000
1/1 [==============================] - 0s 14ms/step - loss: 120613.6719 - val_loss: 58361.0742
Epoch 964/10000
1/1 [==============================] - 0s 14ms/step - loss: 119886.1328 - val_loss: 57784.3711
Epoch 965/10000
1/1 [==============================] - 0s 14ms/step - loss: 151452.5938 - val_loss: 59960.9414
Epoch 966/10000
1/1 [==============================] - 0s 14ms/step - loss: 129924.8516 - val_loss: 52678.9688
E

1/1 [==============================] - 0s 14ms/step - loss: 108091.8672 - val_loss: 53857.2812
Epoch 1032/10000
1/1 [==============================] - 0s 15ms/step - loss: 122020.5234 - val_loss: 60222.2500
Epoch 1033/10000
1/1 [==============================] - 0s 14ms/step - loss: 100365.5078 - val_loss: 64306.1562
Epoch 1034/10000
1/1 [==============================] - 0s 14ms/step - loss: 142171.1875 - val_loss: 58338.9102
Epoch 1035/10000
1/1 [==============================] - 0s 14ms/step - loss: 129962.7812 - val_loss: 62097.0625
Epoch 1036/10000
1/1 [==============================] - 0s 14ms/step - loss: 106371.7812 - val_loss: 64188.5508
Epoch 1037/10000
1/1 [==============================] - 0s 14ms/step - loss: 126181.7812 - val_loss: 52451.7695
Epoch 1038/10000
1/1 [==============================] - 0s 14ms/step - loss: 123353.6641 - val_loss: 52381.7461
Epoch 1039/10000
1/1 [==============================] - 0s 14ms/step - loss: 154482.5000 - val_loss: 66704.3672
Epoch 104

In [19]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[4089.333](test_target) - [[3339.2993]](prediction) = 750.0336835937501


In [20]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [21]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,-0.713127,1.867639,-1.651063,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,...,1.892700,1.196754,-1.592002,-1.354851,-1.049860,-0.866184,-1.563415,-1.661916,-1.819268,-1.744222
1,-0.741803,1.018077,-1.103880,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,...,1.381297,-0.123420,-1.634945,-1.545783,-1.085114,0.273089,-1.054585,-1.113781,-1.110478,-1.068353
2,-0.730617,0.721910,-0.595965,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,...,0.116268,1.087108,1.192449,-0.458462,-1.131090,1.537519,-0.590157,-0.626867,-0.535461,-0.528616
3,-0.497967,-0.327566,-0.126085,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,...,-0.320128,0.820278,0.842482,-0.164403,-0.552517,0.945137,-0.152932,-0.022712,0.051840,-0.012460
4,-0.178038,-0.625198,0.306996,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,...,-0.613535,0.416753,0.336915,0.381074,0.376580,0.401252,0.239109,0.585694,0.525953,0.401855
5,-0.003254,-0.972715,0.704512,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,...,-0.763192,-0.700871,0.541269,1.030697,1.001838,0.004141,0.569273,0.465685,0.821488,0.660648
6,0.426386,-0.622978,1.067697,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,...,-0.869050,-1.793922,0.477106,1.279992,1.300669,-0.397987,0.811321,0.850453,0.830229,0.749280
7,2.438420,-1.059170,1.397787,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,...,-0.824360,-0.902680,-0.163275,0.831736,1.139495,-1.896967,1.741386,1.523443,1.235697,1.541868


train_target:


,Santa Catarina - Consumo de Cimento (t)
0,1740.105
1,1824.650
2,1870.478
3,2081.839
4,2653.587
5,2669.400
6,2852.759
7,3087.810


test_input:


...

,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
17,1.271362,-1.092682,-0.11741,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.364725,0.273868,0.225803,-0.51059,-0.762042,-0.252919,0.886688,-0.271847,-0.02587,0.728476


test_target:


,Santa Catarina - Consumo de Cimento (t)
17,4089.333


1/1 [==============================] - 0s 34ms/step
Error: 750.0336835937501




[2968.81591796875,
 2981.888671875,
 3262.271484375,
 3326.248046875,
 3169.801513671875,
 2965.33447265625,
 3173.48681640625,
 2881.197265625,
 3041.92724609375,
 3339.29931640625]

In [22]:
display(targets)
display(predictions)

[3268.31,
 3466.227,
 3246.749285714286,
 3017.084228571429,
 2776.217485714286,
 2891.489,
 2976.485,
 3258.027,
 3537.032,
 4089.333]

[2968.81591796875,
 2981.888671875,
 3262.271484375,
 3326.248046875,
 3169.801513671875,
 2965.33447265625,
 3173.48681640625,
 2881.197265625,
 3041.92724609375,
 3339.29931640625]

In [23]:
mae = mean_absolute_error(predictions, targets)
mae

339.4917916015624

In [24]:
porcentage = mae/np.mean(targets)
porcentage

0.10437245110672289